In [1]:
# Import dependencies
import chromedriver_autoinstaller
chromedriver_autoinstaller.install()
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import re # to work with regex
import time # to add delays between requests
from datetime import date # to save today's date as 'scraped_date' in data
from pathlib import Path


## Variables that change each run

Please only edit these variables when scraping different pages, to keep output file names consistent.

In [2]:
# Property type code:
# 1 condos, 2 freehold townhomes, 3 condo townhomes, 5 detached homes, 4 no filter(don't use)
property_type_code = 3
# Page range to scrape (inclusive)
first_page_to_scrape = 1
last_page_to_scrape = 15

## Define functions

In [3]:
# Define function to scrape a single page,
# @arg html: scraped html
# @return list of listing dictionaries
def get_listing_info(html):
    # Create a BeautifulSoup object from the scraped HTML
    soup = BeautifulSoup(html, 'html.parser')
    # List of listings in the html soup, found using class sl
    listings_html = soup.find_all('div', 'sl')
    # Empty list to store listing dicts
    listings_list = []
    for item in listings_html:
        # Create a dictionary containing the info for a listing
        listing = {
            'url': item.find('div', 'slt_address').a['href'],
            'address': item.find('div', 'slt_address').text,
            'price': item.find('div', 'slt_price').text.strip('$ CAD').replace(',', ''),
            'baths': item.find('div', 'slt_baths').text.rstrip(' baths'),
            'beds': item.find('div', 'slt_beds').text.rstrip(' beds').partition('+')[0],
            'dens': item.find('div', 'slt_beds').text.rstrip(' beds').partition('+')[2],
            'street': item.find_all('na2')[2].text,
            'neighbourhood': item.find_all('na2')[1].text
        }
        # if no den, put 0
        listing['dens'] = '0' if listing['dens'] == '' else listing['dens']
        # Extract mls_id from url
        listing['mls_id'] = re.search('[A-Z]\\d{7}', listing['url']).group()
        # Append to the list
        listings_list.append(listing)
    # Return expanded listings_list
    return listings_list

In [4]:
# Define function that scrapes a range of pages from listing.ca toronto section
# @arg property_type: 1 is condos, 2 freehold townhomes, 3 condo townhomes, 4 no filter, 5 detached homes
# @return dataframe of listing data
def get_df_from_pages(start_page, end_page, property_type):
    # Define city code (Toronto is cy)
    city = 'cy'
    # Empty listing info list
    listing_info_list = []
    # Launch browser
    driver = webdriver.Chrome()
    # Loop tthrough a range corresponding to search pages and extend listing info list
    for i in range(start_page, end_page + 1):
        url = f'https://listing.ca/mls/?{property_type}.{city}.........{i}..$'
        driver.get(url)
        listing_info_list.extend(get_listing_info(driver.page_source))
        # Wait 1 second, to avoid connection errors
        time.sleep(1)
    # Quit browsing session
    driver.quit()
    # Make df with list of dictionaries
    df = pd.DataFrame(listing_info_list)
    # Dictionary with what city codes mean
    city_code_decoder = {
        'cy': 'Toronto'
    }
    # Add city column
    df['city'] = city_code_decoder[city]
    # Dictionary with what property type codes mean
    property_type_decoder = {
        1: 'condo_apartment',
        2: 'freehold _townhome',
        3: 'condo_townhome',
        5: 'detached_home'
    }
    # Add property type column
    df['property_type'] = property_type_decoder[property_type]
    # Add date scraped colum
    df['date_scraped'] = date.today()
    # Change type of these columns to integer
    for col_name in ['price', 'baths', 'beds', 'dens']:
        df[col_name] = df[col_name].astype('int64')
    return df

## Run scraping function and save results

In [5]:
# Run function with arguments: start and end page and property type code
scraped_df = get_df_from_pages(
    start_page=first_page_to_scrape,
    end_page=last_page_to_scrape,
    property_type=property_type_code
)
# Check result
print(scraped_df.info())
scraped_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   url            40 non-null     object
 1   address        40 non-null     object
 2   price          40 non-null     int64 
 3   baths          40 non-null     int64 
 4   beds           40 non-null     int64 
 5   dens           40 non-null     int64 
 6   street         40 non-null     object
 7   neighbourhood  40 non-null     object
 8   mls_id         40 non-null     object
 9   city           40 non-null     object
 10  property_type  40 non-null     object
 11  date_scraped   40 non-null     object
dtypes: int64(4), object(8)
memory usage: 3.9+ KB
None


,url,address,price,baths,beds,dens,street,neighbourhood,mls_id,city,property_type,date_scraped
0,https://toronto.listing.ca/2335-sheppard-ave-9...,2335 Sheppard Ave 9,649000,2,2,0,Sheppard Ave,Humberlea-Pelmo Park W5,W8031036,Toronto,condo_townhome,2024-01-30
1,https://toronto.listing.ca/715-lawrence-ave-29...,715 Lawrence Ave 29,589900,2,2,0,Lawrence Ave,Yorkdale-Glen Park,W8031032,Toronto,condo_townhome,2024-01-30
2,https://toronto.listing.ca/100-mornelle-crt-20...,100 Mornelle Crt 2032,549900,2,2,1,Mornelle Crt,Morningside,E8030972,Toronto,condo_townhome,2024-01-30
3,https://toronto.listing.ca/36-birch-ave-109.C8...,36 Birch Ave 109,3595000,3,2,0,Birch Ave,Yonge-St. Clair,C8031012,Toronto,condo_townhome,2024-01-30
4,https://toronto.listing.ca/1017-college-st.C80...,1017 College St,999900,3,3,0,College St,Little Portugal,C8030690,Toronto,condo_townhome,2024-01-30


In [7]:
# Save to csv, using args in file name
scraped_df.to_csv(
    Path(
        'data',
        f'listings_type{property_type_code}_' +
        f'pages{first_page_to_scrape:03}-{last_page_to_scrape:03}_{date.today()}.csv'),
    header=True, index=False
)